프로그래밍 언어는 개발자의 관점에 영향을 미치므로 언어의 선택은 중요하다. 우리가 정의한 패턴은 스몰토크나 C++ 수준의 언어 기능을 가정하고 있는데, 언어의 선택은 쉽게 구현할 수 있는 것과 그렇지 않은 것을 결정짓는다. 만약 절차적 언어를 가정했다면 , '상속', '캡슐화', '다형성'이라고 하는 디자인 패턴을 추가 했을 것이다. 마찬가지로 우리가 정의한 패턴 중 일부는 덜 보편화된 일부 객체지향 언어에서 직접적으로 지원할 수 있다. 예를 들어 CLOS는 다중메서드를 지원하므로 비지터(Visitor)와  같은 패턴의 필요성이 줄어든다.

### 6.1 사례: 전략 패턴의 리팩토링
#### 6.1.1 고전적인 전략
- 콘텍스트 : 일부 계산을 서로 다른 알고리즘을 구현하는 교환 가능한 컴포넌트에 위임함으로써 서비스를 제공한다. 전자상거래 예제에서 콘텍스트는 Order로서, 여러 알고리즘 중 하나에 따라 프로모션 할인을 적용하도록 설정한다.
- 전략 : 여러 알고리즘을 구현하는 컴포넌트에 공통된 인터페티스, 전자상거래 예제에서는 이 역할을 Promotion이라는 추상 클래스가 담당한다.
- 구체적인 전략 : 전략의 구상 서브클래스 중 하나. 여기서는 FidelityPromo, BulkItemPromo, LargeOrderPromo등 총 3개의 구체적인 전략이 구현되어 있다.

In [7]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple ('Customer', 'name fidelity')

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity

class Order: # 콘텍스트
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total =  sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total : {:.2f} due: {:.2f}'
        return fmt.format(self.total(),self.due())

class FidelityPromo(Promotion): #첫번째 구체적인 전략
    ''' 충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용'''
    def discount(self, order):
        return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion): #두번째 구체적인 전략
    '''20개 이상의 동일 상품을 구입하면 10%할인 적용'''
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() *.1
        return discount

class LargeOrderPromo(Promotion) : #세번째 구체적인 전략
    '''10종류 이상의 상품을 구입하면 전체 7% 할인 적용'''
    def discount(self, order):
        distint_items = (items.product for item in order.cart)
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0

In [14]:
joe = Customer('John Doe', 0)
#ann = Customer('Ann Smith', 1100)
#cart = [LineItem('banana', 4, .5), LineItem('apple', 10,1.5), LineItem('watermellon', 5, 5.0)]
Order(joe, cart, FidelityPromo)


AttributeError: 'Order' object has no attribute 'cart'

In [15]:
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')
class LineItem:
    def __init__(self, product, quantity, price):
        self.product =  product
        self.quantity = quantity
        self.price = price
    def total(self):
        return self.price * self.quantity

class Order: #콘텍스트
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cuart = list(cart)
        self.promotion = promotion
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

def fidelity_promo(order):
    """충성도 포인트가 1000점 이상인 고객에게 전체 5%할인 적용"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantitiy >= 20:
            discount += item.total()*.1
    return discount

def large_order_promo(order):
    """10종류 이상의 상춤을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total()*0.07
    return 0

In [16]:
joe = Customer('John Doe', 0)
#ann = Customer('Ann Smith', 1100)
#cart = [LineItem('banana', 4, .5), LineItem('apple', 10,1.5), LineItem('watermellon', 5, 5.0)]
Order(joe, cart, FidelityPromo)


AttributeError: 'Order' object has no attribute 'cart'